##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Guía completa de entrenamiento con reconocimiento de la cuantización

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/model_optimization/guide/quantization/training_comprehensive_guide.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

Le damos la bienvenida a la guía completa para el entrenamiento con reconocimiento de la cuantización de Keras.

En esta página, se documentan varios casos de uso y se muestra cómo usar la API para cada uno. Una vez que sepa qué API necesita, busque los parámetros y los detalles de bajo nivel en los [documentos de la API](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/quantization):

- Si quiere ver los beneficios del entrenamiento con reconocimiento de la cuantización y lo que se admite, consulte la [descripción general](https://www.tensorflow.org/model_optimization/guide/quantization/training.md).
- Para ver un ejemplo único de principio a fin, consulte el [ejemplo de entrenamiento con reconocimiento de la cuantización](https://www.tensorflow.org/model_optimization/guide/quantization/training_example.md).

Se tratan los siguientes casos de uso:

- Implementar un modelo con cuantización de 8 bits con estos pasos.
    - Definir un modelo con reconocimiento de la cuantización.
    - Solo para los modelos Keras HDF5, usar una lógica de deserialización y puntos de verificación especiales. De lo contrario, el entrenamiento es estándar.
    - Crear un modelo cuantizado a partir del modelo con reconocimiento de la cuantización.
- Experimentar con la cuantización.
    - Cualquier cosa destinada a la experimentación no tiene una ruta compatible para su implementación.
    - Las capas personalizadas de Keras se encuentran bajo experimentación.

## Preparación

Para encontrar las API que necesita y comprender los propósitos, puede ejecutar, pero omitir la lectura de esta sección.

In [ ]:
! pip install -q tensorflow
! pip install -q tensorflow-model-optimization

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(20, input_shape=input_shape),
      tf.keras.layers.Flatten()
  ])
  return model

def setup_pretrained_weights():
  model= setup_model()

  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )

  model.fit(x_train, y_train)

  _, pretrained_weights = tempfile.mkstemp('.tf')

  model.save_weights(pretrained_weights)

  return pretrained_weights

def setup_pretrained_model():
  model = setup_model()
  pretrained_weights = setup_pretrained_weights()
  model.load_weights(pretrained_weights)
  return model

setup_model()
pretrained_weights = setup_pretrained_weights()

##Definir modelo con reconocimiento de la cuantización

Al definir los modelos de las siguientes maneras, hay rutas disponibles para la implementación en los backends que se enumeran en la [página de descripción general](https://www.tensorflow.org/model_optimization/guide/quantization/training.md). De forma predeterminada, se usa la cuantización de 8 bits.

Nota: un modelo con reconocimiento de la cuantización en realidad no está cuantizado. Crear un modelo cuantizado es un paso aparte.

### Cuantizar todo el modelo

**Su caso de uso:**

- No se admiten modelos con subclases.

**Consejos para una mejor precisión del modelo:**

- Pruebe "Quantize some layers" (cuantizar algunas capas) para omitir la cuantización de las capas que reducen más la precisión.
- Por lo general, es mejor realizar ajustes con cuantización que con el entrenamiento desde cero.


Para que todo el modelo sea con reconocimiento de la cuantización, aplique `tfmot.quantization.keras.quantize_model` al modelo.


In [ ]:
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.summary()

### Cuantizar algunas capas

Cuantizar un modelo puede tener un efecto negativo en la precisión. Puede cuantizar selectivamente capas de un modelo para explorar el equilibrio entre precisión, velocidad y tamaño del modelo.

**Su caso de uso:**

- Para implementar en un backend que solo funciona bien con modelos totalmente cuantizados (por ejemplo, EdgeTPU v1, la mayoría de los DSP), intente "Cuantizar todo el modelo".

**Consejos para una mejor precisión del modelo:**

- Por lo general, es mejor realizar ajustes con cuantización que con el entrenamiento desde cero.
- Pruebe cuantizar las capas posteriores en lugar de las primeras.
- Evite cuantizar capas críticas (por ejemplo, mecanismo de atención).


En el siguiente ejemplo, cuantice sólo las capas `Dense`.

In [ ]:
# Create a base model
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy

# Helper function uses `quantize_annotate_layer` to annotate that only the 
# Dense layers should be quantized.
def apply_quantization_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.quantization.keras.quantize_annotate_layer(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_quantization_to_dense` 
# to the layers of the model.
annotated_model = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_quantization_to_dense,
)

# Now that the Dense layers are annotated,
# `quantize_apply` actually makes the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)
quant_aware_model.summary()

Si bien en este ejemplo se usó el tipo de capa para decidir qué cuantizar, la forma más fácil de cuantizar una capa en particular es establecer su propiedad `name` y buscar ese nombre en `clone_function`.

In [ ]:
print(base_model.layers[0].name)

#### Precisión del modelo más legible pero potencialmente menor

Esto no es compatible con el ajuste con entrenamiento con reconocimiento de la cuantización, por lo que puede ser menos preciso que los ejemplos anteriores.

**Ejemplo funcional**

In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
i = tf.keras.Input(shape=(20,))
x = tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(10))(i)
o = tf.keras.layers.Flatten()(x)
annotated_model = tf.keras.Model(inputs=i, outputs=o)

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

# For deployment purposes, the tool adds `QuantizeLayer` after `InputLayer` so that the
# quantized model can take in float inputs instead of only uint8.
quant_aware_model.summary()

**Ejemplo secuencial**


In [ ]:
# Use `quantize_annotate_layer` to annotate that the `Dense` layer
# should be quantized.
annotated_model = tf.keras.Sequential([
  tfmot.quantization.keras.quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=input_shape)),
  tf.keras.layers.Flatten()
])

# Use `quantize_apply` to actually make the model quantization aware.
quant_aware_model = tfmot.quantization.keras.quantize_apply(annotated_model)

quant_aware_model.summary()

## Establecer punto de verificación y deserializar

**Su caso de uso:** este código solo es necesario para el formato del modelo HDF5 (no para pesos HDF5 u otros formatos).

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Save or checkpoint the model.
_, keras_model_file = tempfile.mkstemp('.h5')
quant_aware_model.save(keras_model_file)

# `quantize_scope` is needed for deserializing HDF5 models.
with tfmot.quantization.keras.quantize_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

## Crear e implementar un modelo cuantizado

En general, consulte la documentación del backend de implementación que usará.

Este es un ejemplo para el backend de TFLite.

In [ ]:
base_model = setup_pretrained_model()
quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)

# Typically you train the model here.

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

quantized_tflite_model = converter.convert()

## Experimentar con la cuantización

**Su caso de uso**: usar las siguientes API significa que no existe una ruta compatible para la implementación. Por ejemplo, la conversión a TFLite y las implementaciones del núcleo solo admiten la cuantización de 8 bits. Las funciones también son experimentales y no están sujetas a compatibilidad con versiones anteriores.

- `tfmot.quantization.keras.QuantizeConfig`
- `tfmot.quantization.keras.quantizers.Quantizer`
- `tfmot.quantization.keras.quantizers.LastValueQuantizer`
- `tfmot.quantization.keras.quantizers.MovingAverageQuantizer`

### Preparación: DefaultDenseQuantizeConfig

Experimentar requiere el uso de `tfmot.quantization.keras.QuantizeConfig`, que describe cómo cuantizar pesos, activaciones y salidas de una capa.

A continuación se muestra un ejemplo que define el mismo `QuantizeConfig` que se usó para la capa `Dense` en los valores predeterminados de la API.

Durante la propagación hacia adelante en este ejemplo, el `LastValueQuantizer` que se devuelve en `get_weights_and_quantizers` se llama con `layer.kernel` como entrada, lo que produce una salida. La salida reemplaza `layer.kernel` en la propagación hacia adelante original de la capa `Dense`, a través de la lógica definida en `set_quantize_weights`. La misma idea se aplica a las activaciones y salidas.


In [ ]:
LastValueQuantizer = tfmot.quantization.keras.quantizers.LastValueQuantizer
MovingAverageQuantizer = tfmot.quantization.keras.quantizers.MovingAverageQuantizer

class DefaultDenseQuantizeConfig(tfmot.quantization.keras.QuantizeConfig):
    # Configure how to quantize weights.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=8, symmetric=True, narrow_range=False, per_axis=False))]

    # Configure how to quantize activations.
    def get_activations_and_quantizers(self, layer):
      return [(layer.activation, MovingAverageQuantizer(num_bits=8, symmetric=False, narrow_range=False, per_axis=False))]

    def set_quantize_weights(self, layer, quantize_weights):
      # Add this line for each item returned in `get_weights_and_quantizers`
      # , in the same order
      layer.kernel = quantize_weights[0]

    def set_quantize_activations(self, layer, quantize_activations):
      # Add this line for each item returned in `get_activations_and_quantizers`
      # , in the same order.
      layer.activation = quantize_activations[0]

    # Configure how to quantize outputs (may be equivalent to activations).
    def get_output_quantizers(self, layer):
      return []

    def get_config(self):
      return {}

### Cuantizar capa de Keras personalizada


En este ejemplo se usa `DefaultDenseQuantizeConfig` para cuantizar `CustomLayer`.

La aplicación de la configuración es la misma en todos los casos de uso de "Experimentar con cuantización".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` a `CustomLayer` y pase `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar cuantizando el resto del modelo con los valores predeterminados de la API.


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class CustomLayer(tf.keras.layers.Dense):
  pass

model = quantize_annotate_model(tf.keras.Sequential([
   quantize_annotate_layer(CustomLayer(20, input_shape=(20,)), DefaultDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `DefaultDenseQuantizeConfig` with `quantize_scope`
# as well as the custom Keras layer.
with quantize_scope(
  {'DefaultDenseQuantizeConfig': DefaultDenseQuantizeConfig,
   'CustomLayer': CustomLayer}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Modificar parámetros de cuantización


**Error común:** cuantizar el sesgo a menos de 32 bits suele perjudicar demasiado la precisión del modelo.

Este ejemplo modifica la capa `Dense` para usar 4 bits para sus pesos en lugar de los 8 bits predeterminados. El resto del modelo sigue usando los valores predeterminados de la API.


In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      return [(layer.kernel, LastValueQuantizer(num_bits=4, symmetric=True, narrow_range=False, per_axis=False))]

La aplicación de la configuración es la misma en todos los casos de uso de "Experimentar con cuantización".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` a la capa `Dense` y pase `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar cuantizando el resto del modelo con los valores predeterminados de la API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Modificar partes de la capa para cuantizar


Este ejemplo modifica la capa `Dense` para omitir la cuantización de la activación. El resto del modelo sigue usando los valores predeterminados de la API.

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    def get_activations_and_quantizers(self, layer):
      # Skip quantizing activations.
      return []

    def set_quantize_activations(self, layer, quantize_activations):
      # Empty since `get_activaations_and_quantizers` returns
      # an empty list.
      return

La aplicación de la configuración es la misma en todos los casos de uso de "Experimentar con cuantización".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` a la capa `Dense` y pase `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar cuantizando el resto del modelo con los valores predeterminados de la API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this Dense layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()

### Usar un algoritmo de cuantización personalizado


La clase `tfmot.quantization.keras.quantizers.Quantizer` es una clase invocable que puede aplicar cualquier algoritmo a sus entradas.

En este ejemplo, las entradas son los pesos y aplicamos las matemáticas en la función __call__ `FixedRangeQuantizer` a los pesos. En lugar de los valores originales de los pesos, la salida de `FixedRangeQuantizer` ahora se pasa a lo que hubieran usado los pesos.

In [ ]:
quantize_annotate_layer = tfmot.quantization.keras.quantize_annotate_layer
quantize_annotate_model = tfmot.quantization.keras.quantize_annotate_model
quantize_scope = tfmot.quantization.keras.quantize_scope

class FixedRangeQuantizer(tfmot.quantization.keras.quantizers.Quantizer):
  """Quantizer which forces outputs to be between -1 and 1."""

  def build(self, tensor_shape, name, layer):
    # Not needed. No new TensorFlow variables needed.
    return {}

  def __call__(self, inputs, training, weights, **kwargs):
    return tf.keras.backend.clip(inputs, -1.0, 1.0)

  def get_config(self):
    # Not needed. No __init__ parameters to serialize.
    return {}


class ModifiedDenseQuantizeConfig(DefaultDenseQuantizeConfig):
    # Configure weights to quantize with 4-bit instead of 8-bits.
    def get_weights_and_quantizers(self, layer):
      # Use custom algorithm defined in `FixedRangeQuantizer` instead of default Quantizer.
      return [(layer.kernel, FixedRangeQuantizer())]

La aplicación de la configuración es la misma en todos los casos de uso de "Experimentar con cuantización".

- Aplique `tfmot.quantization.keras.quantize_annotate_layer` a la capa `Dense` y pase `QuantizeConfig`.
- Use `tfmot.quantization.keras.quantize_annotate_model` para continuar cuantizando el resto del modelo con los valores predeterminados de la API.

In [ ]:
model = quantize_annotate_model(tf.keras.Sequential([
   # Pass in modified `QuantizeConfig` to modify this `Dense` layer.
   quantize_annotate_layer(tf.keras.layers.Dense(20, input_shape=(20,)), ModifiedDenseQuantizeConfig()),
   tf.keras.layers.Flatten()
]))

# `quantize_apply` requires mentioning `ModifiedDenseQuantizeConfig` with `quantize_scope`:
with quantize_scope(
  {'ModifiedDenseQuantizeConfig': ModifiedDenseQuantizeConfig}):
  # Use `quantize_apply` to actually make the model quantization aware.
  quant_aware_model = tfmot.quantization.keras.quantize_apply(model)

quant_aware_model.summary()